In [2]:
import pandas as pd
import numpy as np

In [3]:
df = pd.DataFrame({'From_To': ['LoNDon_paris', 'MAdrid_miLAN',
'londON_StockhOlm',

'Budapest_PaRis', 'Brussels_londOn'],
'FlightNumber': [10045, np.nan, 10065, np.nan, 10085],
'RecentDelays': [[23, 47], [], [24, 43, 87], [13], [67, 32]],
'Airline': ['KLM(!)', '<Air France> (12)', '(British Airways. )',
'12. Air France', '"Swiss Air"']})

In [4]:
df

,From_To,FlightNumber,RecentDelays,Airline
0,LoNDon_paris,10045.0,"[23, 47]",KLM(!)
1,MAdrid_miLAN,NaN,[],<Air France> (12)
2,londON_StockhOlm,10065.0,"[24, 43, 87]",(British Airways. )
3,Budapest_PaRis,NaN,[13],12. Air France
4,Brussels_londOn,10085.0,"[67, 32]","""Swiss Air"""


#### 1. Some values in the the FlightNumber column are missing. These numbers are meant to increase by 10 with each row so 10055 and 10075 need to be put in place. Fill in these missing numbers and make the column an integer column (instead of a float column).

In [30]:
new_index=np.arange(1,df.From_To.count()+1)
df.set_index(new_index,inplace=True)
df

,From_To,FlightNumber,RecentDelays,Airline
1,LoNDon_paris,10045.0,"[23, 47]",KLM(!)
2,MAdrid_miLAN,NaN,[],<Air France> (12)
3,londON_StockhOlm,10065.0,"[24, 43, 87]",(British Airways. )
4,Budapest_PaRis,NaN,[13],12. Air France
5,Brussels_londOn,10085.0,"[67, 32]","""Swiss Air"""


In [42]:
for i in np.arange(1,df.From_To.count()+1):
    if pd.isnull(df.FlightNumber.loc[i]):
        df.FlightNumber.loc[i,"FlightNumber"]=df.FlightNumber.loc[i-1]+10
df["FlightNumber"].astype(int)
df
        

,From_To,FlightNumber,RecentDelays,Airline
1,LoNDon_paris,10045,"[23, 47]",KLM(!)
2,MAdrid_miLAN,10055,[],<Air France> (12)
3,londON_StockhOlm,10065,"[24, 43, 87]",(British Airways. )
4,Budapest_PaRis,10075,[13],12. Air France
5,Brussels_londOn,10085,"[67, 32]","""Swiss Air"""


In [44]:
df.dtypes

From_To         object
FlightNumber     int32
RecentDelays    object
Airline         object
dtype: object

#### 2. The From_To column would be better as two separate columns! Split each string on the underscore delimiter _ to give a new temporary DataFrame with the correct values. Assign the correct column names to this temporary DataFrame.

In [60]:
tempdf=df.copy()
tempdf[["From","To"]]= tempdf.From_To.str.split("_",expand=True)
tempdf

,From_To,FlightNumber,RecentDelays,Airline,From,To
1,LoNDon_paris,10045,"[23, 47]",KLM(!),LoNDon,paris
2,MAdrid_miLAN,10055,[],<Air France> (12),MAdrid,miLAN
3,londON_StockhOlm,10065,"[24, 43, 87]",(British Airways. ),londON,StockhOlm
4,Budapest_PaRis,10075,[13],12. Air France,Budapest,PaRis
5,Brussels_londOn,10085,"[67, 32]","""Swiss Air""",Brussels,londOn


#### 3. Notice how the capitalisation of the city names is all mixed up in this temporary DataFrame. Standardise the strings so that only the first letter is uppercase (e.g. "londON" should become "London".)

In [69]:
tempdf["From"]=tempdf["From"].str.capitalize()
tempdf["To"]=tempdf["To"].str.capitalize()
tempdf["From_To"]=tempdf["From_To"].str.capitalize()

tempdf

,From_To,FlightNumber,RecentDelays,Airline,From,To
1,London_paris,10045,"[23, 47]",KLM(!),London,Paris
2,Madrid_milan,10055,[],<Air France> (12),Madrid,Milan
3,London_stockholm,10065,"[24, 43, 87]",(British Airways. ),London,Stockholm
4,Budapest_paris,10075,[13],12. Air France,Budapest,Paris
5,Brussels_london,10085,"[67, 32]","""Swiss Air""",Brussels,London


#### 4. Delete the From_To column from df and attach the temporary DataFrame from the previous questions.

In [71]:
df

,FlightNumber,RecentDelays,Airline
1,10045,"[23, 47]",KLM(!)
2,10055,[],<Air France> (12)
3,10065,"[24, 43, 87]",(British Airways. )
4,10075,[13],12. Air France
5,10085,"[67, 32]","""Swiss Air"""


In [73]:
df["From_To"]=tempdf["From_To"]
df

,FlightNumber,RecentDelays,Airline,From_To
1,10045,"[23, 47]",KLM(!),London_paris
2,10055,[],<Air France> (12),Madrid_milan
3,10065,"[24, 43, 87]",(British Airways. ),London_stockholm
4,10075,[13],12. Air France,Budapest_paris
5,10085,"[67, 32]","""Swiss Air""",Brussels_london


#### 5. In the RecentDelays column, the values have been entered into the DataFrame as a list. We would like each first value in its own column, each second value in its own column, and so on. If there isn't an Nth value, the value should be NaN.
Expand the Series of lists into a DataFrame named delays, rename the columns
delay_1, delay_2, etc. and replace the unwanted RecentDelays column in df
with delays.

In [74]:
delays=df["RecentDelays"].apply(pd.Series)

In [75]:
delays

,0,1,2
1,23.0,47.0,NaN
2,NaN,NaN,NaN
3,24.0,43.0,87.0
4,13.0,NaN,NaN
5,67.0,32.0,NaN


In [76]:
delays.columns=["delay_{}".format(n) for n in range(1,len(delays.columns)+1)]

In [77]:
delays.columns

Index(['delay_1', 'delay_2', 'delay_3'], dtype='object')

In [78]:
df = df.drop('RecentDelays', axis=1).join(delays)

df

,FlightNumber,Airline,From_To,delay_1,delay_2,delay_3
1,10045,KLM(!),London_paris,23.0,47.0,NaN
2,10055,<Air France> (12),Madrid_milan,NaN,NaN,NaN
3,10065,(British Airways. ),London_stockholm,24.0,43.0,87.0
4,10075,12. Air France,Budapest_paris,13.0,NaN,NaN
5,10085,"""Swiss Air""",Brussels_london,67.0,32.0,NaN
